includes everything except words analysis, 6 hidden nodes, deep learning less acurate, loss= categorical_crossentropy

without following and followers

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("final_output.csv", encoding = "latin1")

In [3]:
df.head()

,label,id,name,username,bio,location,url,join_date,join_time,tweets,...,media,private,verified,profile_image_url,background_image,color_number,face_detection,follower/following_ratio,b_key_count,i_key_count
0,individual,379823034,mohammed,BarcaMohammed,NaN,North Bergen,NaN,9/25/11,9:33:00,172.0,...,1.0,0.0,0.0,https://pbs.twimg.com/profile_images/166601617...,NaN,51912.0,0.0,0.023256,0.0,0.0
1,individual,931229576,SaMisches,Shmisch,NaN,NaN,NaN,11/6/12,20:41:00,6.0,...,3.0,0.0,0.0,https://pbs.twimg.com/profile_images/930857677...,NaN,31033.0,0.0,0.108696,0.0,0.0
2,individual,393095898,Christopher Pawelski,ChrisPawelski,A 4th generation family onion farmer married t...,"Florida, NY",http://about.me/chris_pawelski,10/17/11,18:00:00,49503.0,...,23400.0,0.0,0.0,https://pbs.twimg.com/profile_images/686658233...,https://pbs.twimg.com/profile_banners/39309589...,46205.0,0.0,1.085431,0.0,1.0
3,individual,3109060051,xrishin,xrishin,NaN,NaN,NaN,3/26/15,19:39:00,3.0,...,0.0,1.0,0.0,https://abs.twimg.com/sticky/default_profile_i...,NaN,184.0,0.0,0.050000,0.0,0.0
4,business,1.13E+18,LENDVER,LendverLLC,BORROW CONFIDENTLY. We__Î¢_öÎå_Î¢ve do...,NaN,https://www.lendver.com/,5/29/19,7:32:00,2.0,...,1.0,0.0,0.0,https://pbs.twimg.com/profile_images/113375859...,https://pbs.twimg.com/profile_banners/11337428...,510.0,0.0,0.090909,0.0,0.0


In [4]:
df.shape

(11456, 23)

In [5]:
# 999 was a code we used when executing python face_detection library on twitter profile pictures. 
# Specifically, 999 indicated that there was an error/no picture found when iterating through thr profile pictures
df = df[df.face_detection != 999]
# code 1 was assigned if a face was deteced, code 0 was assigned if a face was not detected.

In [6]:
# Create a column in dataframe indicating whether a url was linked on the twitter page or not. 
# If a url was detected, code 1 was assigned, else, 0 was assigned.
array = []
for i in list(df["url"]):
    if str(i) != "nan":
        array.append(1)
    else:
        array.append(0)
df["url_detect"] = array

In [7]:
df.shape

(11264, 24)

In [8]:
# color_number column is a column that gives a count of the number of colors a profile picture has.
# We noticed that pictures with 184 colors all linked to the same default twitter profile picture.
# Since no face recognition or insightful color count can be returned and properly evaluated by the model,
# we decided to drop all rows that have a color count of 184.
# Coincidentally, we noticed color_number = 0 all corresponding face_detection = 999.
# Even though that was already dropped, we decided to add this precaution in case this was used on future scraped data
df = df[(df.color_number != 184) & (df.color_number !=0)]

In [9]:
df.shape

(10467, 24)

In [10]:
# Drop columns we dont need for modeling
# df = df.drop(columns = ["media", "name", "following", "followers", "id", "url", "bio", "location", "join_date", "join_time", \
#                         "private", "verified", "profile_image_url", "background_image", "i_key_count"])
df = df[["label", "username", "face_detection", "color_number", "b_key_count", "i_key_count", "url_detect"]]

In [11]:
df = df.dropna(how="any")
# Dataset used had 11401 rows, unclassified as business or not. Our team went into each twitter page to classify
# whether the page was a business or individual. Since 1000s of rows could not be manually classified, 
# we dropped many unclassified rows
# Dropping data must be the last step, because there are blank fields in the dropped columns, which may have values
# in the variables we will take into consideration. Dropping pre-emptively will drop many rows unnecessarily

In [12]:
df.shape

(2357, 7)

In [13]:
df.head()

,label,username,face_detection,color_number,b_key_count,i_key_count,url_detect
0,individual,BarcaMohammed,0.0,51912.0,0.0,0.0,0
1,individual,Shmisch,0.0,31033.0,0.0,0.0,0
2,individual,ChrisPawelski,0.0,46205.0,0.0,1.0,1
4,business,LendverLLC,0.0,510.0,0.0,0.0,1
6,individual,ClaretSport,1.0,41897.0,0.0,0.0,1


In [14]:
# Assign X (data) and y (target)
X = df.drop(["label", "username"], axis=1)
y = df["label"]
print(X.shape, y.shape)

(2357, 5) (2357,)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [24]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier
classifier.fit(X_train, y_train)

# Prediction results on test data set
predictions = classifier.predict(X_test)
comparison = pd.DataFrame({"prediction": predictions, "actual": y_test})
comparison.head()

/Users/kyleeng/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,prediction,actual
10300,individual,business
1404,individual,individual
10284,individual,individual
2494,individual,business
1935,individual,individual


In [22]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6468590831918506
Testing Data Score: 0.6474576271186441


In [17]:
# joined dataset so we can see usernames and predictions/actual side by side
merged_comparsion = df.join(comparison, how ="inner")
merged_comparsion = merged_comparsion[["username", "prediction", "actual"]]

merged_comparsion.head()

,username,prediction,actual
2,ChrisPawelski,individual,individual
10,Arvidinvest,individual,individual
17,pattyshort,individual,individual
21,HarshnaParoha,individual,individual
23,jsare34,individual,individual


In [19]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_train, y_train)

0.9954725523486134

In [20]:
# Get weights that machine defined, multiplied by 100 to get percents
importances = rf.feature_importances_*100
importances

array([29.74523888, 54.15080507,  7.30441884,  1.68413465,  7.11540257])

In [21]:
sorted(zip(importances, X), reverse=True)

[(54.15080506755916, 'color_number'),
 (29.745238875813087, 'face_detection'),
 (7.304418836684552, 'b_key_count'),
 (7.115402565934826, 'url_detect'),
 (1.6841346540083724, 'i_key_count')]